In [1]:
import re
import datetime
import csv
import sys 
import codecs
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from helpers import *

,0,1,2,3,4,5,6,7,8
pf10_y40,834,CIUDAD DE MÉXICO,COYOACAN,F,35,20/03/2020,Sospechoso,Contacto,NA
pf10_y41,835,HIDALGO,TEPEJI DEL RIO,F,24,23/03/2020,Sospechoso,Contacto,NA
pf10_y9d,887,AGUASCALIENTES,AGUASCALIENTES,F,59,23/03/2020,Sospechoso,Contacto,NA
pf10_ya,839,OAXACA,VALLES CENTRALES Y CAÑADA,F,51,23/03/2020,Sospechoso,Contacto,NA
pf10_ya0,888,CIUDAD DE MÉXICO,BENITO JUAREZ,F,35,15/03/2020,Sospechoso,Contacto,NA
...,...,...,...,...,...,...,...,...,...
pff_y99,829,BAJA CALIFORNIA,TIJUANA,M,28,11/03/2020,Sospechoso,Estados Unidos,09/03/2020
pff_y9b,830,AGUASCALIENTES,RINCON DE ROMOS,F,65,17/03/2020,Sospechoso,Estados Unidos,10/03/2020
pff_y9c,831,SAN LUIS POTOSÍ,SAN LUIS POTOSI,F,42,21/03/2020,Sospechoso,Contacto,NA
pff_y9d,832,OAXACA,VALLES CENTRALES Y CAÑADA,F,21,19/03/2020,Sospechoso,Contacto,NA


['Caso',
 'Estado',
 'Localidad',
 'Sexo',
 'Edad',
 'Fecha_Sintomas',
 'Situacion',
 'Procedencia',
 'Fecha_Llegada']

In [65]:
df.columns = get_col_names(df)

df = df.drop(df[df['Caso'].str.contains('Caso', regex=True, na=False)].index)



KeyError: 0

In [67]:
df['Estado_Normalizado'] = [get_iso_from_state(normalize_text(x)) for x in df.Estado]

if 'Localidad' in df.columns:
    df['Localidad_Normalizado'] = [normalize_text(x) for x in df.Localidad]


df['Sexo_Normalizado'] = [normalize_sex(x) for x in df.Sexo]
df['Fecha_Sintomas_Normalizado'] = [normalize_date(get_fixed_date(x)) for x in df.Fecha_Sintomas]
df['Situacion_Normalizado'] = [normalize_text(x) for x in df.Situacion]
df['Fecha_Sintomas_Corregido'] = [is_date_in_excel_format(x) for x in df.Fecha_Sintomas]

if 'Fecha_Llegada' in df.columns:
    df['Fecha_Llegada'] = [normalize_date(get_fixed_date(x)) for x in df.Fecha_Llegada]
    
if 'Procedencia' in df.columns:
    df['Procedencia_Normalizado'] = [normalize_origin(x) for x in df.Procedencia]

#np.unique(df.Fecha_Sintomas_Corregido, return_counts=True)
df

,Caso,Estado,Localidad,Sexo,Edad,Fecha_Sintomas,Situacion,Procedencia,Fecha_Llegada,Estado_Normalizado,Localidad_Normalizado,Sexo_Normalizado,Fecha_Sintomas_Normalizado,Situacion_Normalizado,Fecha_Sintomas_Corregido,Procedencia_Normalizado
pf10_y40,834,CIUDAD DE MÉXICO,COYOACAN,F,35,20/03/2020,Sospechoso,Contacto,NA,CMX,COYOACAN,F,2020-03-20,SOSPECHOSO,False,CONTACTO
pf10_y41,835,HIDALGO,TEPEJI DEL RIO,F,24,23/03/2020,Sospechoso,Contacto,NA,HID,TEPEJI DEL RIO,F,2020-03-23,SOSPECHOSO,False,CONTACTO
pf10_y9d,887,AGUASCALIENTES,AGUASCALIENTES,F,59,23/03/2020,Sospechoso,Contacto,NA,AGU,AGUASCALIENTES,F,2020-03-23,SOSPECHOSO,False,CONTACTO
pf10_ya,839,OAXACA,VALLES CENTRALES Y CAÑADA,F,51,23/03/2020,Sospechoso,Contacto,NA,OAX,VALLES CENTRALES Y CAÑADA,F,2020-03-23,SOSPECHOSO,False,CONTACTO
pf10_ya0,888,CIUDAD DE MÉXICO,BENITO JUAREZ,F,35,15/03/2020,Sospechoso,Contacto,NA,CMX,BENITO JUAREZ,F,2020-03-15,SOSPECHOSO,False,CONTACTO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pff_y99,829,BAJA CALIFORNIA,TIJUANA,M,28,11/03/2020,Sospechoso,Estados Unidos,2020-03-09,BCN,TIJUANA,M,2020-03-11,SOSPECHOSO,False,ESTADOS UNIDOS
pff_y9b,830,AGUASCALIENTES,RINCON DE ROMOS,F,65,17/03/2020,Sospechoso,Estados Unidos,2020-03-10,AGU,RINCON DE ROMOS,F,2020-03-17,SOSPECHOSO,False,ESTADOS UNIDOS
pff_y9c,831,SAN LUIS POTOSÍ,SAN LUIS POTOSI,F,42,21/03/2020,Sospechoso,Contacto,NA,SLP,SAN LUIS POTOSI,F,2020-03-21,SOSPECHOSO,False,CONTACTO
pff_y9d,832,OAXACA,VALLES CENTRALES Y CAÑADA,F,21,19/03/2020,Sospechoso,Contacto,NA,OAX,VALLES CENTRALES Y CAÑADA,F,2020-03-19,SOSPECHOSO,False,CONTACTO


In [68]:
df.to_csv(destination, index = False)
